In [12]:
#### Run 1 - file baru perlu text to column
import pandas as pd
import numpy as np
import os
import re
import csv

selcen = pd.DataFrame()
for i in os.listdir('Settlement Project\Blibli NutriMart\Seller Centre = warehouse'):
    if '$' not in i:
        if 'csv' in i:
            temp = pd.read_csv('Settlement Project\Blibli NutriMart\Seller Centre = warehouse/' + str(i), skipinitialspace=True)
            if 'rusak' in i or '202103' in i or '202104' in i or '202105' in i or '202107' in i or '202108' in i or '202009' in i or '202010' in i or '202011' in i or '5 sd 31 Aug' in i:                
                names= temp.columns
                temp2 = pd.DataFrame()
                for line in csv.reader(temp['No. Order'], skipinitialspace=True):
                    temp2 = temp2.append(pd.DataFrame(np.reshape(line, (1,16))), ignore_index = True, sort = False)
                temp2.columns = names
                temp = temp2.copy()
            temp['Source'] = i
            for j in ['No. Order', 'No. Order Item', 'No. Awb', 'Kode SKU', 'Merchant SKU']:
                temp[j] = temp[j].astype(str).str.replace('"', '', regex = False).str.replace('=', '', regex = False)
            true_col = ['No. Order', 'No. Order Item', 'No. Awb', 'Tanggal Order', 'Nama Pemesan', 'Kode SKU', 'Blibli SKU', 'Merchant SKU', 'Nama Produk', 'Total Barang',
                        'Harga Produk', 'Servis Logistik', 'Kode Merchant', 'Nama Store', 'Order Status', 'Toko/Gudang', 'Source']
            print(i)
            if temp.columns.to_list() == true_col :
                print('Column sama')
            else:
                print('Column beda')
                print([x for x in true_col if x not in temp.columns.to_list()])
                print([x for x in temp.columns.to_list() if x not in true_col])
            if 'No Order' in selcen.columns:
                temp=temp[~temp['No. Order'].isin(selcen['No. Order'].unique())]
            selcen = selcen.append(temp, ignore_index = True, sort = False)

bulk-order-download-template 202201.csv
Column sama
bulk-order-download-template 202202.csv
Column sama
bulk-order-download-template 202203.csv
Column sama
bulk-order-download-template 202204.csv
Column sama
bulk-order-download-template 202205.csv
Column sama
bulk-order-download-template 202206.csv
Column beda
[]
['Unnamed: 0']
bulk-order-download-template 202207.csv
Column sama
bulk-order-download-template 202208.csv
Column sama
bulk-order-download-template 202209.csv
Column sama
bulk-order-download-template 202210.csv
Column sama
bulk-order-download-template 202211.csv
Column sama
bulk-order-download-template 202212.csv
Column sama
bulk-order-download-template 202301.csv
Column sama
bulk-order-download-template 202302.csv
Column sama
bulk-order-download-template 202303.csv
Column sama
bulk-order-download-template 202304.csv
Column sama
bulk-order-download-template 202305.csv
Column sama
bulk-order-download-template 202306.csv
Column sama
bulk-order-download-template 202307.csv
Column

In [4]:
i

'bulk-order-download-template 202407.csv'

In [15]:
### run 2, must null, not null = duplicate
selcen[selcen[['No. Order','No. Order Item', 'Kode SKU', 'Blibli SKU', 'Merchant SKU', 'Nama Produk', 'Total Barang']].duplicated(keep = False)].sort_values(['No. Order Item'])#['No. Order Item'].nunique()

,No. Order,No. Order Item,No. Awb,Tanggal Order,Nama Pemesan,Kode SKU,Blibli SKU,Merchant SKU,Nama Produk,Total Barang,...,Nama diskon,Kode diskon,Voucher seller,Nama voucher seller,Kode voucher seller,Nama voucher gratis ongkir,Kode voucher gratis ongkir,Catatan produk,Tanggal pengiriman,Tanggal pengiriman;


In [14]:
#### optional if there is duplicates, check to forstok/sellcen data blibli sering duplicate
selcen = selcen.drop_duplicates(['No. Order','No. Order Item', 'Kode SKU', 'Blibli SKU', 'Merchant SKU', 'Nama Produk'], keep = 'last')

In [19]:
## run 3 
selcen = selcen.rename(columns = {'Merchant SKU' : 'SKU', 'Total Barang' : 'Qty. Invoiced'})
selcen['Qty. Invoiced'] = selcen['Qty. Invoiced'].astype(int)

selcen['Date'] = np.nan
selcen['Month'] = np.nan
selcen['Year'] = np.nan

### tambahan timo April 23
selcen['true datetime']=np.nan

indeks = selcen[pd.to_datetime(selcen['Tanggal Order']).dt.strftime('%d').astype(int) <= 12].index.to_list()
selcen['Date'][indeks] = pd.to_datetime(pd.to_datetime(selcen['Tanggal Order'][indeks]).dt.strftime('%Y-%d-%m %H:%M')).dt.day
selcen['Month'][indeks] = pd.to_datetime(pd.to_datetime(selcen['Tanggal Order'][indeks]).dt.strftime('%Y-%d-%m %H:%M')).dt.month_name()
selcen['Year'][indeks] = pd.to_datetime(pd.to_datetime(selcen['Tanggal Order'][indeks]).dt.strftime('%Y-%d-%m %H:%M')).dt.year

### tambahan timo April 23
selcen['true datetime'][indeks] = pd.to_datetime(pd.to_datetime(selcen['Tanggal Order'][indeks]).dt.strftime('%Y-%d-%m %H:%M'))

non_indeks = selcen[~selcen.index.isin(indeks)].index.to_list()
selcen['Date'][non_indeks] = pd.to_datetime(selcen['Tanggal Order'][non_indeks]).dt.day
selcen['Month'][non_indeks] = pd.to_datetime(selcen['Tanggal Order'][non_indeks]).dt.month_name()
selcen['Year'][non_indeks] = pd.to_datetime(selcen['Tanggal Order'][non_indeks]).dt.year

### tambahan timo April 23
selcen['true datetime'][non_indeks] = pd.to_datetime(selcen['Tanggal Order'][non_indeks])

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [20]:
### pastiin bulan realisasi masuk
selcen[['Year', 'Month']].drop_duplicates()
selcen.groupby(['Year','Month'])[['No. Order']].nunique().reset_index()

,Year,Month,No. Order
0,2022.0,April,2408
1,2022.0,August,2955
2,2022.0,December,2053
3,2022.0,February,2134
4,2022.0,January,2423
5,2022.0,July,3158
6,2022.0,June,1866
7,2022.0,March,2228
8,2022.0,May,2022
9,2022.0,November,2548


In [21]:
### run 4
selcen_new = selcen[selcen['Year'] > 2022]

In [11]:
cek_dups = selcen_new[(selcen_new[['No. Order', 'SKU']].duplicated(keep = False)) & (selcen_new['SKU'].notnull())].groupby('SKU')['Nama Produk'].unique().reset_index()
( cek_dups.join( pd.DataFrame(cek_dups['Nama Produk'].to_list())
                 .add_prefix('array_') )
        .drop('Nama Produk', axis = 1) )#.values


,SKU,array_0,array_1,array_2,array_3,array_4,array_5,array_6
0,(B)2106323165,Gift - Tropicana Slim Bumbu Saus Telur Asin 3 ...,None,None,None,None,None,None
1,(E)2104406,Tropicana Slim Minyak Sunflower 946ml - Minyak...,None,None,None,None,None,None
2,(E)2104406P2,Twin Pack -Tropicana Slim Minyak Sunflower 946...,None,None,None,None,None,None
3,(E)2106023014P3,WHS - Triple Pack - Tropicana Slim Shirataki N...,None,None,None,None,None,None
4,(E)2HF1403250P4,Harga Spesial - 4 Pack - HiLo School Cotton Ca...,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
95,2T01705024,Tropicana Slim French Butter Souffle Coffee 10...,None,None,None,None,None,None
96,2T01705024P2,Twin Pack - Tropicana Slim French Butter Souff...,None,None,None,None,None,None
97,2T03201001,Tropicana Slim Bumbu Kaldu Ayam dan Jamur 100 ...,None,None,None,None,None,None
98,PT23T24,Paket Coffee Tropicana Slim: Cafe Latte 10 sac...,None,None,None,None,None,None


In [12]:
np.unique([len(x) for x in cek_dups['Nama Produk']])

array([1, 2, 7])

In [13]:
print('ok')

ok


In [22]:
### run 5
temp2 = selcen_new.copy()

import time
import pandas as pd
import numpy as np
import math

import smtplib 
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from smtplib import SMTP
import smtplib
import sys
import requests
import os
    
data_SKU = pd.read_excel(r"D:\1. Masterdata\SKU_File\data_SKU.xlsx",engine='openpyxl')

s = requests.Session()
s.get("https://tatanama.pythonanywhere.com")
s.post("https://tatanama.pythonanywhere.com", data = {'username' : 'ecommerce', 'password' : 'ttnmecom24'})
r = s.get("https://tatanama.pythonanywhere.com/download")

with open(r"D:\1. Masterdata\SKU_File\Master tatanama.xlsx", 'wb') as output:
    output.write(r.content)

if os.path.isfile(r"D:\1. Masterdata\SKU_File\Master tatanama.xlsx") :    
    SKU_append = pd.read_excel(r"D:\1. Masterdata\SKU_File\Master tatanama.xlsx")
    SKU_append.columns = [x.replace('_', ' ') for x in SKU_append.columns]
    data_SKU = data_SKU[~data_SKU['SKU'].astype(str).isin(SKU_append['SKU'].astype(str))]
    data_SKU = data_SKU.append(SKU_append, ignore_index = True, sort = False)

to_excel = data_SKU.to_excel(r"D:\1. Masterdata\SKU_File\data_SKU.xlsx", index = False)
data_SKU=SKU_append.copy()

In [9]:
selcen.loc[selcen['Nama Produk']=='Tropicana Slim Almond Drink Chocolicious 190ml (1 pcs) - Minuman Almond Ready to Drink Rasa Cokelat ']['SKU'].unique()

array([], dtype=object)

In [23]:
### run 7 , harus null, not null = cek sku & perbaiki
selcen_new.loc[selcen_new['SKU'].str.contains(" "),'SKU']=selcen_new['SKU'].str.strip()
selcen_new.loc[selcen_new['SKU']=='P2101151180P2','SKU']='PT7(2)B65'
selcen_new.loc[selcen_new['SKU']=='21014281P4','SKU']="2HF1401250P4"
selcen_new.loc[selcen_new['SKU']=='21014281P24','SKU']="2HF1401250P24"
selcen_new.loc[selcen_new['Nama Produk']=='WHS - Paket Spesial: Tropicana Slim Hokkaido Cheese Cookies 100g & Chocolate Spread 300g ','SKU']='PT40T41'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - HiLo Active Ketan Hitam 175gr - Susu Tinggi Kalsium','SKU']='2101445144'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - NutriSari Ready to Drink Mango Smoothie (24 tetrapack) - Minuman Buah Vitamin C','SKU']='1101155250P24'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - Lokalate Kopi Berondong 10 Sachet - Kopi Sachet Kopi Kekinian','SKU']='1101656318'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - Twin Pack - HiLo Teen Taro 500g - Susu Tinggi Kalsium Lebih Rendah Lemak','SKU']='2101609180P2'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - Paket Spesial: Tropicana Slim Hokkaido Cheese Cookies 100g & Chocolate Spread 300g','SKU']='PT40T41'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - Paket Rendah Kalori - Tropicana Slim Hokkaido Cheese Cookies & Sweetener I Sweet','SKU']='PT19T40'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - Paket NutriSari Jeruk Peras, HiLo Es Ketan Hitam, dan HiLo Klepon Latte Powder Drink Refill 500 gram','SKU']='PH81H82N60'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - FREE HILO CHOCOLATE AVOCADO RTD - Twin Pack : HiLo Teen Popcorn Caramel 500 gram - Susu Tinggi Kalsium Lebih Rendah Lemak ','SKU']='PH67(2)B111'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - FREE HILO CHOCOLATE AVOCADO RTD - HiLo Teen Popcorn Caramel 500 gram - Susu Tinggi Kalsium Lebih Rendah Lemak ','SKU']='PH67B111'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - HiLo Milky Brown Sugar Ready to Drink 200ml (24 tetrapack) - Minuman Sumber Kalsium Rendah Lemak','SKU']='2101941250P24'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - Twin Pack - Tropicana Slim Shirataki Noodle: Mie Goreng [71 g]','SKU']='2106023014P2'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - HiLo School Chocolate Ready to Drink 200ml (4 tetrapack) - Susu Tinggi Kalsium Rendah Lemak','SKU']='2101492P4'
selcen_new.loc[selcen_new['Nama Produk']=='HiLo School Vanilla Vegiberi Ready To Drink 200ml x 1pcs','SKU']='2HF1401250'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - HiLo Kacang Hijau Ready to Drink 200ml (24 tetrapack) - Minuman Sumber Kalsium Rendah Lemak','SKU']='2101907250P24'
selcen_new.loc[selcen_new['Nama Produk']=='Tropicana Slim Hampers Lebaran Idul Fitri Fancy Tote Bag','SKU']='PT24T51T52T61T79G292G294'
selcen_new.loc[selcen_new['Nama Produk']=='DISKON BOMBASTIS - Tropicana Slim Almond Drink Chocolicious 190ml (1 pcs) - Minuman Almond Ready to Drink Rasa Cokelat ','SKU']='2T01402249'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - NutriSari Es Rujak Jeruk Bali 40sch - Minuman Buah Vitamin C Vitamin D','SKU']='(E)1101996453'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - NutriSari RTD Squeezed Orange Minuman Jeruk Peras Vitamin C 200 mL ','SKU']='1N01402250P4'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - HiLo Teen Taro 500 gr - Susu Tinggi Kalsium','SKU']='2101609180'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - HiLo Chocolate Avocado Ready to Drink 200ml (4 Tetrapack) - Susu Siap Minum Tinggi Kalsium','SKU']='2HH1407250P4'
selcen_new.loc[selcen_new['Nama Produk']=='TG - HiLo School Chocolate RTD 200ml','SKU']='2101492'
selcen_new.loc[selcen_new['Nama Produk']=='TG - Tropicana Slim Almond Drink Chocolicious 190ml (1 pcs) - Minuman Almond Ready to Drink Rasa Cokelat ','SKU']='2T01402249'

selcen_new.loc[selcen_new['Nama Produk']=='Tropicana Slim Oat Drink Japanese Cantaloupe Melon 190 ml (1 Pcs) - Minuman Oat Vegan Plant Based Bebas Gula Ready to Drink','SKU']='2T01403249'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - HiLo Gold Biscuit Cereal 500g - Susu Tinggi Kalsium Lebih Rendah Lemak','SKU']='2101524180'
selcen_new.loc[selcen_new['Nama Produk']=='Nutrifood - BBF Mystery Box','SKU']='PH100H108(2)T71'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - HiLo Teen Chocolate Ready to Drink (4 pcs) - Susu Tinggi Kalsium Rendah Lemak','SKU']='2101461P4'

selcen_new.loc[selcen_new['Nama Produk']=='TG - HiLo School Chocolate RTD 200ml ','SKU']='2101461P4'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - Paket HiLo School - HiLo School Vanilla Vegiberi 500g & Original 400g','SKU']='PH4H102'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - Twin Pack - HiLo Gold Biscuit Cereal 500g - Susu Tinggi Kalsium Lebih Rendah Lemak','SKU']='2101524180P2'

selcen_new.loc[selcen_new['Nama Produk']=='TG - HiLo Protein Chocofit UHT 190 mL (1 pcs) – Susu Tinggi Protein dengan Collagen Rasa Cokelat','SKU']='2HH1409249'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - Paket School 3+ - HiLo School 3+ Strawberry Pop 400g & School 3+ Soya Vanilla Malt 400g','SKU']='PH100H101'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - HiLo Gold Susu Tinggi Kalsium Rendah Lemak Chocolate [500 g ]','SKU']='2101551180'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - Twin Pack - HiLo School 3+ Strawberry Pop 400g','SKU']='2HF0113021P2'

selcen_new.loc[selcen_new['Nama Produk']=='WHS - Twin Pack : Tropicana Slim Bumbu Kaldu Ayam dan Jamur 100 gram - lebih rendah garam dan tanpa MSG','SKU']='2T03201001P2'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - Paket HiLo Dessert - HiLo Es Ketan Hitam, Es Teler, Swiss Chocolate, Thai Tea & Pisang Ijo - Minuman Bubuk Tinggi Kalsium','SKU']='PH23H28H62H63H88'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - Tropicana Slim Sweetener Lemongrass Pandan 50 sch - Bantu Batasi Gula','SKU']='2102526125'
selcen_new.loc[selcen_new['Nama Produk']=="FREE Gift - W'dank Bajigur 4 Sachet ",'SKU']='(B)1101665104'

selcen_new.loc[selcen_new['Nama Produk']=='Tropicana Slim Sweetener Classic 100 sachet - Pemanis untuk Bantu Batasi Gula ','SKU']='2102500130'
selcen_new.loc[selcen_new['Nama Produk']=='Tropicana Slim Susu Skim Original 1kg - Bantu Turunkan Kolesterol ','SKU']='2101000195'
selcen_new.loc[selcen_new['Nama Produk']=='Paket HiLo Teen - HiLo Teen Vanilla Caramel 500g & Popcorn Caramel 500g - Susu Tinggi Kalsium Lebih Rendah Lemak ','SKU']='PH35H67'
selcen_new.loc[selcen_new['Nama Produk']=='WHS - HiLo Thai Tea Ready to Drink 200ml x 4pcs - Minuman Sumber Kalsium','SKU']='2101947250P4'

selcen_new.loc[selcen_new['Nama Produk']=='Tropicana Slim Susu Skim Coffee [500 g x 2 pcs] ','SKU']='2101150180'
selcen_new.loc[selcen_new['Nama Produk']=='Tropicana Slim Susu Skim Coffee 500g - Bantu Turunkan Kolesterol ','SKU']='2101150180'
selcen_new.loc[selcen_new['Nama Produk']=='Tropicana Slim Sweetener Diabtx 100 Sachet - Pemanis untuk Diabetesi ','SKU']='2102900130'
selcen_new.loc[selcen_new['Nama Produk']=='Hilo School Vanilla Vegiberi Minuman Kesehatan [500 gr] ','SKU']='2101428180'
selcen_new.loc[selcen_new['Nama Produk']=='EYD - HiLo Teen Chocolate - Susu Tinggi Kalsium - 1000gr','SKU']='2101651195'
selcen_new.loc[selcen_new['Nama Produk']=='Tropicana Slim Almond Drink Chocolicious 190ml (1 pcs) - Minuman Almond Ready to Drink Rasa Cokelat','SKU']='2T01402249'

selcen_new.loc[selcen_new['Nama Produk']=='NutriSari Es Kuwud Nipis 40 Sachet - Minuman Buah Vitamin C Vitamin D','SKU']='1101595453'
selcen_new.loc[selcen_new['Nama Produk']=='Gift - Tropicana Slim French Butter Souffle Coffee 4 Sachet','SKU']='(B)2T01705164'
selcen_new.loc[selcen_new['Nama Produk']=='Gift - Tropicana Slim French Butter Souffle Coffee 4 Sachet ','SKU']='(B)2T01705164'
selcen_new.loc[selcen_new['Nama Produk']=='HiLo Yoghurt Smoothie Bowl Strawberry 8 Sachet - Yoghurt Sumber Kalsium dan Vitamin ','SKU']='2101457108'

selcen_new[~selcen_new['SKU'].isin(data_SKU['SKU'])][['No. Order', 'Nama Produk', 'SKU']].drop_duplicates('Nama Produk').values

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


array([], shape=(0, 3), dtype=object)

In [16]:
print(selcen_new[selcen_new['Nama Produk'].str.contains('HiLo Yoghurt Smoothie Bowl Strawberry 8 Sachet - Yoghurt Sumber Kalsium dan Vitamin ')]['SKU'].unique())


['2101457108']


array([], shape=(0, 3), dtype=object)

In [24]:
## run 7.1
from datetime import datetime,timedelta
listjanjian=requests.get('http://event.l-men.com/apiv1?type=janjian')
for i in listjanjian.json():
    sku=i['realsku']
    print(sku)
    start=datetime.fromtimestamp(int(i['startdate']/1000)).replace(hour=0)
    end=datetime.fromtimestamp(int(i['enddate']/1000)).replace(hour=23)
    print(i['notes'])
    selcen_new.loc[(selcen_new['SKU']==sku)&(selcen_new['true datetime']>=start)&(selcen_new['true datetime']<end),'Pemilik Budget']=i['notes']
    print('=====')
    # print(str(datetime.fromtimestamp(int(i['enddate']/1000)).replace(hour=0)))

2T00803164
Launching tahap 2
=====
2T00803164P3
Launching tahap 2


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)


=====
2T02703229P2
Launching
=====
PT25T74
Launching
=====
2T02703229
Launching tahap 2
=====
2104115110
Launching tahap 2
=====
2104115110P2
Launching tahap 2
=====
2104148024
Launching tahap 2
=====
2104148024P2
Launching tahap 2
=====
1N01402250P24
Marketing
=====
1N01402250P4
Marketing
=====
2104010270
Marketing
=====
1101909331
Marketing
=====
2T00803164P2
Marketing
=====
2T00803164P3
Marketing
=====
2HF0114021
Launching tahap 2
=====
2101751112
Marketing
=====
2101751112P2
Marketing
=====
2101700166P2
Marketing
=====
2101700166
Marketing
=====
2101384106
Internal
=====
2101384106P3
Internal
=====
2105055180P2
Internal
=====
2104523105P3
Internal
=====
2104508105P3
Internal
=====
2104543105P3
Internal
=====
2104589105P3
Internal
=====
2104241210P2
Internal
=====
2106386249P24
Internal
=====
2101000195
Internal
=====
2101151180
Internal
=====
2101150180
Internal
=====
2101200180
Internal
=====
2102500125
Internal
=====
2102500130
Internal
=====
2102500125P2
Internal
=====
210250013

In [25]:
### run 8
# Merging with SKU Attributes
print('== Merging with SKU Attributes ==')

temp2 = selcen_new.copy()

temp2 = temp2.rename(columns = {'Nama Produk' : 'Product Name'})

data_SKU.loc[data_SKU['Price List NFI']=='-','Price List NFI']=0
data_SKU['Price List NFI'] = data_SKU['Price List NFI'].astype(float).astype('int64')
data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)#.str.replace('(S)', '', regex = False)
data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)
list_col = ['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand']

temp_merge = temp2.merge(data_SKU[list_col].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp2.index)
for i in list_col:
    temp2[i] = temp_merge[str(i)]

list_col = ['SKU'] + data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Cost 7')+1].to_list()
temp_merge = temp2.merge(data_SKU[list_col].drop_duplicates(['SKU']), how = 'left', left_on = 'Real SKU', right_on = 'SKU').set_index(temp2.index)
list_pcs = [x for x in temp_merge.columns if 'PCS' in x]
for i in list_pcs:
    temp_merge[i] = temp_merge[i] * temp_merge['Qty. Invoiced']

list_col = data_SKU.columns[data_SKU.columns.get_loc('Produk 1'):data_SKU.columns.get_loc('Harga Cost 7')+1].to_list()    
for i in list_col:
    temp2[i] = temp_merge[str(i)]
    
print('== Unbundling ==')
data_bundle1 = temp2[~temp2['Produk 1'].isnull()]
data_bundle1['Bundle Name'] = data_bundle1['Product Name']
data_bundle1['Product Name'] = data_bundle1['Produk 1']
data_bundle1['SKU'] = data_bundle1['SKU Produk 1']
data_bundle1['Qty. Invoiced'] = data_bundle1['PCS Produk 1']
data_bundle1['Price List NFI'] = data_bundle1['Price List NFI 1']
data_bundle1['Total Net'] = data_bundle1['Price List NFI 1']
data_bundle1['Bundle Flag'] = np.nan

data_bundle2 = temp2[~temp2['Produk 2'].isnull()]
data_bundle2['Bundle Name'] = data_bundle2['Product Name']
data_bundle2['Product Name'] = data_bundle2['Produk 2']
data_bundle2['SKU'] = data_bundle2['SKU Produk 2']
data_bundle2['Qty. Invoiced'] = data_bundle2['PCS Produk 2']
data_bundle2['Price List NFI'] = data_bundle2['Price List NFI 2']
data_bundle2['Total Net'] = data_bundle2['Price List NFI 2'] 
data_bundle2['Bundle Flag'] = np.nan

data_bundle3 = temp2[~temp2['Produk 3'].isnull()]
data_bundle3['Bundle Name'] = data_bundle3['Product Name']
data_bundle3['Product Name'] = data_bundle3['Produk 3']
data_bundle3['SKU'] = data_bundle3['SKU Produk 3']
data_bundle3['Qty. Invoiced'] = data_bundle3['PCS Produk 3']
data_bundle3['Price List NFI'] = data_bundle3['Price List NFI 3']
data_bundle3['Total Net'] = data_bundle3['Price List NFI 3'] 
data_bundle3['Bundle Flag'] = np.nan

data_bundle4 = temp2[~temp2['Produk 4'].isnull()]
data_bundle4['Bundle Name'] = data_bundle4['Product Name']
data_bundle4['Product Name'] = data_bundle4['Produk 4']
data_bundle4['SKU'] = data_bundle4['SKU Produk 4']
data_bundle4['Qty. Invoiced'] = data_bundle4['PCS Produk 4']
data_bundle4['Price List NFI'] = data_bundle4['Price List NFI 4']
data_bundle4['Total Net'] = data_bundle4['Price List NFI 4'] 
data_bundle4['Bundle Flag'] = np.nan

data_bundle5 = temp2[~temp2['Produk 5'].isnull()]
data_bundle5['Bundle Name'] = data_bundle5['Product Name']
data_bundle5['Product Name'] = data_bundle5['Produk 5']
data_bundle5['SKU'] = data_bundle5['SKU Produk 5']
data_bundle5['Qty. Invoiced'] = data_bundle5['PCS Produk 5']
data_bundle5['Price List NFI'] = data_bundle5['Price List NFI 5']
data_bundle5['Total Net'] = data_bundle5['Price List NFI 5']
data_bundle5['Bundle Flag'] = np.nan

data_bundle6 = temp2[~temp2['Produk 6'].isnull()]
data_bundle6['Bundle Name'] = data_bundle6['Product Name']
data_bundle6['Product Name'] = data_bundle6['Produk 6']
data_bundle6['SKU'] = data_bundle6['SKU Produk 6']
data_bundle6['Qty. Invoiced'] = data_bundle6['PCS Produk 6']
data_bundle6['Price List NFI'] = data_bundle6['Price List NFI 6']
data_bundle6['Total Net'] = data_bundle6['Price List NFI 6']
data_bundle6['Bundle Flag'] = np.nan

data_bundle7 = temp2[~temp2['Produk 7'].isnull()]
data_bundle7['Bundle Name'] = data_bundle7['Product Name']
data_bundle7['Product Name'] = data_bundle7['Produk 7']
data_bundle7['SKU'] = data_bundle7['SKU Produk 7']
data_bundle7['Qty. Invoiced'] = data_bundle7['PCS Produk 7']
data_bundle7['Price List NFI'] = data_bundle7['Price List NFI 7']
data_bundle7['Total Net'] = data_bundle7['Price List NFI 7']
data_bundle7['Bundle Flag'] = np.nan

data_bundle = data_bundle1.append([data_bundle2, data_bundle3, data_bundle4, data_bundle5, data_bundle6, data_bundle7], ignore_index = True, sort = False)
data_bundle['SKU'] = data_bundle['SKU'].astype(str)
data_bundle['SKU'] = data_bundle['SKU'].str.replace('\.0$', '', regex = True)
data_bundle[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']] = data_bundle.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU')[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item', 'Parent SKU']]

print("== Pricing ==")
temp2 = temp2.append(data_bundle, ignore_index = True, sort = False)
list_col = ['Real SKU', 'Price List NFI', 'Harga Cost']

temp_merge = temp2.merge(data_SKU[list_col].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(temp2.index)
for i in list_col:
    if i == 'Real SKU' or i == 'Price List NFI':
        temp2[i] = temp_merge[str(i) + '_y']
    else :
        temp2[i] = temp_merge[str(i)]
    
temp2['Price List NFI'] = pd.to_numeric(temp2['Price List NFI']).astype(int)
temp2['Harga Cost'] = pd.to_numeric(temp2['Harga Cost']).astype(int)
temp2['Qty. Invoiced'] = pd.to_numeric(temp2['Qty. Invoiced']).astype(int)

temp2['Total Net'] = temp2['Price List NFI'] * temp2['Qty. Invoiced']
temp2['Total Harga Cost'] = temp2['Harga Cost'] * temp2['Qty. Invoiced']

selcen_sku = temp2.copy()

# Merging with J Converter
print('== Merging with J Converter ==')
j_conv = pd.read_excel("Settlement Project\Rumus Realisasi.xlsx", sheet_name = 'Paket Brand')
j_conv = j_conv.iloc[:,:5]
j_conv = j_conv.rename(columns = {'SKU' : 'Real SKU', 'SKU.1' : 'SKU Produk'})
j_conv = j_conv[j_conv['Real SKU'].notnull()]
j_conv['SKU Produk'] = j_conv['SKU Produk'].astype(str).str.replace('.0', '',regex = False)

temp_all = selcen_sku.copy()

temp_all['Real SKU'] = temp_all['Real SKU'].astype(str)
j_conv['Real SKU'] = j_conv['Real SKU'].astype(str)

temp_all = temp_all.merge(j_conv, how = 'left', on = 'Real SKU')

paket_j = temp_all[temp_all['SKU Produk'].notnull()]

paket_j['Bundle Name'] = paket_j['Product Name']
paket_j['Product Name'] = paket_j['Name']
paket_j['SKU'] = paket_j['SKU Produk']
paket_j['Qty. Invoiced'] = paket_j['Qty. Invoiced'] * paket_j['Konversi ke UOM'].astype(float)
paket_j['Bundle Flag'] = np.nan
    
data_SKU['Real SKU'] = data_SKU['SKU'].astype(str)
data_SKU['Real Nama Produk'] = data_SKU['Nama Produk'].astype(str)
if 'Real SKU_y' in paket_j.columns:
    paket_j = paket_j.drop('Real SKU_y', axis = 1)
paket_j['SKU'] = paket_j['SKU'].astype(str)
paket_j[['Real SKU', 'Real Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU', 'Price List NFI']] = paket_j.merge(data_SKU[['Real SKU', 'Nama Produk', 'Brand', 'Sub Brand', 'Parent Item', 'Parent SKU', 'Price List NFI']].drop_duplicates(['Real SKU']), how = 'left', left_on = 'SKU', right_on = 'Real SKU').set_index(paket_j.index)[['Real SKU_y', 'Nama Produk', 'Brand_y', 'Sub Brand_y', 'Parent Item_y', 'Parent SKU_y', 'Price List NFI_y']]

paket_j['Total Net'] = paket_j['Price List NFI'] * paket_j['Qty. Invoiced']

paket_j = paket_j.drop(['Nama Paket', 'SKU Produk', 'Name', 'Konversi ke UOM'], axis = 1)
selcen_sku = selcen_sku.append(paket_j, ignore_index = True, sort = False)
indeks = selcen_sku[selcen_sku['Product Name'].isin(paket_j['Bundle Name'])].index.to_list()
selcen_sku['Brand'][indeks] = 'Bundle'

indeks = selcen_sku[(selcen_sku['Real SKU'].astype(str).str.contains('^\(B\)', regex = True)) &
      (~selcen_sku['Brand'].isin(['Bonus Produk', 'Gimmick']))].index.to_list()

selcen_sku['Brand'][indeks] = 'Bonus Produk'

indeks = selcen_sku[(selcen_sku['Real SKU'].astype(str).str.contains('^\(U\)', regex = True)) &
      (~selcen_sku['Brand'].isin(['Bonus Produk', 'Gimmick']))].index.to_list()

selcen_sku['Brand'][indeks] = 'Bonus Produk'

# Selcen Done
print('== Selcen Done ==')

== Merging with SKU Attributes ==
== Unbundling ==


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

== Pricing ==
== Merging with J Converter ==


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

== Selcen Done ==


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [26]:
## run for check, jika ingin di unbundling isi rumus realisasi xlsx sheet paket brand
selcen_sku[(selcen_sku['Real SKU'].str.contains('J'))&~(selcen_sku['Real SKU'].isin(j_conv['Real SKU'].unique()))][['Real SKU','Real Nama Produk']].drop_duplicates()


,Real SKU,Real Nama Produk


In [21]:
temp2[temp2['Price List NFI'].isnull()][['SKU','Real Nama Produk','Product Name']].drop_duplicates()

,SKU,Real Nama Produk,Product Name


In [27]:
## run 8.1
for i in listjanjian.json():
    sku=i['realsku']
    print(sku)
    start=datetime.fromtimestamp(int(i['startdate']/1000)).replace(hour=0)
    end=datetime.fromtimestamp(int(i['enddate']/1000)).replace(hour=23)
    print(i['notes'])
    selcen_sku.loc[(selcen_sku['Pemilik Budget'].isnull())&(selcen_sku['SKU']==sku)&(selcen_sku['true datetime']>=start)&(selcen_sku['true datetime']<end),'Pemilik Budget']=i['notes']
    print('=====')

2T00803164
Launching tahap 2
=====
2T00803164P3
Launching tahap 2
=====
2T02703229P2
Launching
=====
PT25T74
Launching
=====
2T02703229
Launching tahap 2
=====
2104115110
Launching tahap 2
=====
2104115110P2
Launching tahap 2
=====
2104148024
Launching tahap 2
=====
2104148024P2
Launching tahap 2
=====
1N01402250P24
Marketing
=====
1N01402250P4
Marketing
=====
2104010270
Marketing
=====
1101909331
Marketing
=====
2T00803164P2
Marketing
=====
2T00803164P3
Marketing
=====
2HF0114021
Launching tahap 2
=====
2101751112
Marketing
=====
2101751112P2
Marketing
=====
2101700166P2
Marketing
=====
2101700166
Marketing
=====
2101384106
Internal
=====
2101384106P3
Internal
=====
2105055180P2
Internal
=====
2104523105P3
Internal
=====
2104508105P3
Internal
=====
2104543105P3
Internal
=====
2104589105P3
Internal
=====
2104241210P2
Internal
=====
2106386249P24
Internal
=====
2101000195
Internal
=====
2101151180
Internal
=====
2101150180
Internal
=====
2101200180
Internal
=====
2102500125
Internal
===

In [28]:
### run 9
import numpy as np

selcen_sku['Product Name'] = selcen_sku['Product Name'].astype(str).str.strip().str.replace('\\n', '', regex = False).str.replace('\n', '', regex = False)
indeks = selcen_sku[selcen_sku['Bundle Name'].isnull()].index.to_list()
selcen_sku['Bundle Name'] = selcen_sku['Bundle Name'].astype(str).str.strip().str.replace('\\n', '', regex = False).str.replace('\n', '', regex = False)
selcen_sku['Bundle Name'][indeks] = np.nan

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [44]:
### run 10
source = pd.read_csv(r"D:\6. settlement\Code\Settlement Project\Backup\Realisasi Jan 2023\Final Result\Raw Unpivoted\CSV\Runningan Blibli By Selcen Jan 22 - Dec 22 rev J.csv", sep = ';')

# source['True datetime'] = pd.to_datetime(source['True datetime'])
# source = source[(source['Year'] > 2021) & (source['True datetime']<'2022-02-01')]
source = source[(source['Year'] < 2022)]
source['Order #'] = source['No. Order'].astype(str)
# source['Order #'] = source['Order #'].astype(str)

selcen_old = selcen_sku[selcen_sku['No. Order'].isin(source['Order #'])]

source['Product Name'] = source['Product Name'].fillna(source['Real Nama Produk'])

selcen_bundle_null = selcen_old[selcen_old['Bundle Name'].isnull()].index.to_list()
source_bundle_null = source[source['Bundle Name'].isnull()].index.to_list()

selcen_old['Bundle Name'] = selcen_old['Bundle Name'].astype(str).str.strip().str.replace('\\n', '', regex = False).str.replace('\n', '', regex = False)
source['Bundle Name'] = source['Bundle Name'].astype(str).str.strip().str.replace('\\n', '', regex = False).str.replace('\n', '', regex = False)

source['Product Name'] = source['Product Name'].astype(str).str.strip().str.replace('\\n', '', regex = False).str.replace('\n', '', regex = False)

dict_change = {'Buy 12 Get 12 ? HiLo School Chocolate RTD 200ml (ED 6 bulan)':'Buy 12 Get 12 – HiLo School Chocolate RTD 200ml (ED 6 bulan)'}

for key in dict_change:
    selcen_old['Bundle Name'] = selcen_old['Bundle Name'].astype(str).str.replace(key, dict_change[key], regex = False)
    source['Bundle Name'] = source['Bundle Name'].astype(str).str.replace(key, dict_change[key], regex = False)
    
import numpy as np

selcen_old['Bundle Name'][selcen_bundle_null] = np.nan
source['Bundle Name'][source_bundle_null] = np.nan


selcen_old_merge = selcen_old.merge(source[['Order #', 'Real SKU', 'Real Nama Produk', 'Bundle Name', 'Price List NFI', 'Qty. Invoiced']].drop_duplicates(['Order #', 'Real SKU', 'Real Nama Produk', 'Bundle Name', 'Qty. Invoiced']), how = 'left', left_on = ['No. Order', 'Real SKU', 'Real Nama Produk', 'Bundle Name', 'Qty. Invoiced'], right_on = ['Order #', 'Real SKU', 'Real Nama Produk', 'Bundle Name', 'Qty. Invoiced']) 

selcen_old_null = selcen_old_merge[selcen_old_merge['Price List NFI_y'].isnull()]

selcen_old_null = selcen_old_null.drop(['Order #', 'Price List NFI_y'], axis = 1)
selcen_old_null = selcen_old_null.rename(columns = {'Price List NFI_x' : 'Price List NFI'})

selcen_old_null = selcen_old_null.merge(source[['Order #', 'Product Name', 'Bundle Name', 'Price List NFI']].drop_duplicates(['Order #', 'Product Name', 'Bundle Name', 'Price List NFI']), how = 'left', left_on = ['No. Order', 'Product Name',  'Bundle Name'], right_on = ['Order #', 'Product Name', 'Bundle Name']).set_index(selcen_old_null.index) 
selcen_old_merge['Price List NFI_y'][selcen_old_null.index] = selcen_old_null['Price List NFI_y']

selcen_old_null = selcen_old_merge[selcen_old_merge['Price List NFI_y'].isnull()]

source2 = source.copy()

# source2['Real SKU'] = source2['Real SKU'].astype(str).str.replace('2309005305', '2309005300')

# indeks = source2[source2['Real SKU'] == '(B)2101809250'].index.to_list()
# source2['Real SKU'][indeks] = '(U)2101809250'

# indeks = source2[source2['Real SKU'] == 'PH48(3)B35'].index.to_list()
# source2['Real SKU'][indeks] = '2101941250P4'

# indeks = source2[source2['Real SKU'] == 'PH39(3)B17'].index.to_list()
# source2['Real SKU'][indeks] = '2101809250P4'

selcen_old_null = selcen_old_null.drop(['Order #', 'Price List NFI_y'], axis = 1)
selcen_old_null = selcen_old_null.rename(columns = {'Price List NFI_x' : 'Price List NFI'})

selcen_old_null = selcen_old_null.merge(source2[['Order #', 'Real SKU', 'Bundle Name', 'Price List NFI']].drop_duplicates(['Order #', 'Real SKU', 'Bundle Name']), how = 'left', left_on = ['No. Order', 'Real SKU',  'Bundle Name'], right_on = ['Order #', 'Real SKU', 'Bundle Name']).set_index(selcen_old_null.index) 

selcen_old_merge['Price List NFI_y'][selcen_old_null.index] = selcen_old_null['Price List NFI_y']



C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (41,49,56,57,64,65,80,84) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giov

In [25]:
source#['Year'].unique()

,No. Order,Product ID,No. Awb,Order Date Seller Centre,Nama Pemesan,Kode SKU,Blibli SKU,SKU,Product Name,Qty. Invoiced,...,Paid January,Total Produk Prop Adj,BTL Cost,WH,Status Gudang,Duplicated,Paid December,Paid Beda,Unnamed: 0,Order #


In [26]:
selcen_old_merge[selcen_old_merge['Price List NFI_y'].isnull()][['No. Order','Product Name','Real Nama Produk', 'Real SKU','Bundle Name', 'Price List NFI_x', 'Price List NFI_y','Qty. Invoiced']].drop_duplicates('Real SKU')

,No. Order,Product Name,Real Nama Produk,Real SKU,Bundle Name,Price List NFI_x,Price List NFI_y,Qty. Invoiced


In [27]:
### run 11
selcen_old_merge = selcen_old_merge.drop(['Price List NFI_x', 'Order #'], axis = 1)
selcen_old_merge = selcen_old_merge.rename(columns = {'Price List NFI_y' : 'Price List NFI'})

selcen_old_merge['Price List NFI'] = pd.to_numeric(selcen_old_merge['Price List NFI']).astype(float)
selcen_old_merge['Qty. Invoiced'] = pd.to_numeric(selcen_old_merge['Qty. Invoiced']).astype(float)

selcen_old_merge['Total Net'] = selcen_old_merge['Price List NFI'] * selcen_old_merge['Qty. Invoiced']

selcen_sku = selcen_sku[~selcen_sku['No. Order'].isin(selcen_old_merge['No. Order'])]
selcen_sku = selcen_sku.append(selcen_old_merge, ignore_index = True, sort = False)

selcen_sku['Total Net Before PPN'] = selcen_sku['Total Net']/1.11

In [29]:
selcen_sku['Total Net Before PPN'] = selcen_sku['Total Net']/1.11

In [30]:
### run 12
import os
import pandas as pd
col_ren=pd.read_excel(r"Settlement Project\Blibli NutriMart\rename_col_sett.xlsx")#.to_dict(orient='list')
col_ren.set_index('Kolom baru')['Kolom Lama'].to_dict()

settlement = pd.DataFrame()
for i in os.listdir('Settlement Project\Blibli NutriMart\Settlement'):
    if 'xlsx' in i:
        temp = pd.read_excel('Settlement Project\Blibli NutriMart\Settlement/' + str(i))
        if 'Promo shipping (Shipping promo)' in temp.columns:
            temp=temp.drop(columns=['Promo shipping (Shipping promo)'])
        if 'Seller SKU' in temp.columns:
            temp=temp.drop(columns=['Seller SKU'])
        if 'Note' in temp.columns:
            temp=temp.drop(columns=['Note'])
        if 'Source' in temp.columns:
            temp=temp.drop(columns=['Source'])
        if 'Pickup Point Code' in temp.columns:
            temp=temp.drop(columns=['Pickup Point Code','Pickup Point Name'])
        if 'Settlement Code' in temp.columns:
            temp=temp.drop(columns=['Settlement Code'])
        if 'Total Margin' in temp.columns:
            temp['Biaya komisi (Commission fee)']=temp['Total Margin']
            temp=temp.drop(columns=['Regular Margin', 'Special Margin', 'Add On Margin', 'Total Margin'])
        # if temp.columns.to_list() == col_ren['Kolom baru'].to_list():
        if len([x for x in temp.columns.to_list() if x not in col_ren['Kolom baru'].to_list()])==0:
            temp=temp.rename(columns=col_ren.set_index('Kolom baru')['Kolom Lama'].to_dict())
        
        
        temp['Source'] = i
        if 'Description' in temp.columns:
            temp = temp.rename(columns = {'Description' : 'Keterangan',
                                         'Unit Price' : 'Harga Satuan',
                                         'Total Price' : 'Jumlah',
                                          'Merchant Promotion' : 'Promosi Merchant',
                                          'Transaction Fee' : 'Biaya Transaksi'
                                         })
        print(i)
        true_col = ['Trx ID', 'Order ID', 'Order Item ID', 'Keterangan', 'Transaction', 'Delivered Date', 'Qty', 'Harga Satuan', 'Jumlah', 'Promosi Merchant', 'Fee', 
                    'Biaya Transaksi', 'PPH 23', 'Total', 'Trx Source', 'Source']
        # temp=temp[true_col]
        if temp.columns.to_list() == true_col :
            print('Column sama')
        else:
            print('Column beda')
            print([x for x in true_col if x not in temp.columns.to_list()])
            print([x for x in temp.columns.to_list() if x not in true_col])
        if 'Order ID' in settlement.columns:
            temp=temp[~temp['Order Item ID'].isin(settlement['Order Item ID'].unique())]
        # if 12126420077 in temp['Order Item ID'].unique():
        #     print('ada')
        # if 'Order ID' in settlement.columns:
        #     if 12126420077 in settlement['Order Item ID'].unique():
        #         print('ada')
        settlement = settlement.append(temp, ignore_index = True, sort = False)
settlementcp=settlement.copy()

20221231_NUT-60023_SettlementReport.xlsx
Column beda
[]
['Payment Fee Merchant', 'Biaya Kirim Seller']
20230103_NUT-60023_SettlementReport.xlsx
Column beda
[]
['Payment Fee Merchant', 'Biaya Kirim Seller']
20230104_NUT-60023_SettlementReport.xlsx
Column beda
[]
['Payment Fee Merchant', 'Biaya Kirim Seller']
20230105_NUT-60023_SettlementReport.xlsx
Column beda
[]
['Payment Fee Merchant', 'Biaya Kirim Seller']
20230108_NUT-60023_SettlementReport.xlsx
Column beda
[]
['Payment Fee Merchant', 'Biaya Kirim Seller']
20230109_NUT-60023_SettlementReport.xlsx
Column beda
[]
['Payment Fee Merchant', 'Biaya Kirim Seller']
20230110_NUT-60023_SettlementReport.xlsx
Column beda
[]
['Payment Fee Merchant', 'Biaya Kirim Seller']
20230111_NUT-60023_SettlementReport.xlsx
Column beda
[]
['Payment Fee Merchant', 'Biaya Kirim Seller']
20230112_NUT-60023_SettlementReport.xlsx
Column beda
[]
['Payment Fee Merchant', 'Biaya Kirim Seller']
20230115_NUT-60023_SettlementReport.xlsx
Column beda
[]
['Payment Fee Mer

In [37]:
[x for x in temp.columns.to_list() if x not in col_ren['Kolom baru'].to_list()]

['Trx ID',
 'Order ID',
 'Order Item ID',
 'Keterangan',
 'Transaction',
 'Delivered Date',
 'Qty',
 'Harga Satuan',
 'Jumlah',
 'Promosi Merchant',
 'Payment Fee Merchant',
 'Biaya Transaksi',
 'PPH 23',
 'Biaya Kirim Seller',
 'Trx Source',
 'Fee',
 'Source']

In [30]:
### for backup
settlement=settlementcp.copy()

In [31]:
### run 12.1
settlement=settlement[settlement['Trx ID']!='a']
settlement['Order ID']=settlement['Order ID'].astype(str).str.replace('.0', '', regex = False)
settlement['Order Item ID']=settlement['Order Item ID'].astype(str).str.replace('.0', '', regex = False)

In [32]:
### run 12.2 
settdetail = pd.DataFrame()
truecol=['Nomor pesanan (Order ID)', 'Nomor item pesanan (Order item ID)','Nama produk (Product name)', 'Jumlah (Quantity)',
         'Harga satuan (Price per unit)', 'Total', 'Promo seller (Seller promo)','Transaksi', 'Tipe (Type)', 'Nilai margin (Margin Amount)',
         'Margin minimal (Minimum Margin)', 'Margin maksimal (Maximum Margin)','Biaya transaksi (Transaction fee)', 'Sumber (Source)',
         'Tipe Seller (seller type)', 'Dikirim oleh (fulfilled by)','Kategori volume berat (Volume weight category)',
         'Tipe pengiriman (delivery type)', 'Tipe pembayaran (payment type)','Tenor', 'Produk Kategori (category product)', 'Margin ID']
for i in os.listdir('Settlement Project\Blibli NutriMart\Settlement'):
    if 'xlsx' in i:
        # temp = pd.read_excel('Settlement Project\Blibli L-Men\Settlement/' + str(i),sheet_name='SSR Detail')
        if 'SSR Detail' in pd.ExcelFile('Settlement Project\Blibli NutriMart\Settlement/' + str(i)).sheet_names:
            print(i)
            temp = pd.read_excel('Settlement Project\Blibli NutriMart\Settlement/' + str(i),sheet_name='SSR Detail',dtype={'Nomor pesanan (Order ID)': str,'Nomor item pesanan (Order item ID)':str})
            if 'Submission Group' in temp.columns:
                temp=temp.rename(columns={'Submission Group':'Margin ID'})
            if 'Nama Margin (Margin Name)' in temp.columns:
                temp=temp.rename(columns={'Nama Margin (Margin Name)':'Tipe (Type)'})
            colbeda=[i for i in temp.columns if i not in truecol]
            if len(colbeda)>0:
                print(colbeda)
            settdetail = settdetail.append(temp[truecol], ignore_index = True, sort = False)
settdetailcp=settdetail.copy()

20231203_NUT-60023_SettlementReport.xlsx
20231204_NUT-60023_SettlementReport.xlsx
20231205_NUT-60023_SettlementReport.xlsx
20231206_NUT-60023_SettlementReport.xlsx
20231207_NUT-60023_SettlementReport.xlsx
20231210_NUT-60023_SettlementReport.xlsx
20231211_NUT-60023_SettlementReport.xlsx
20231212_NUT-60023_SettlementReport.xlsx
20231213_NUT-60023_SettlementReport.xlsx
20231214_NUT-60023_SettlementReport.xlsx
20231217_NUT-60023_SettlementReport.xlsx
20231218_NUT-60023_SettlementReport.xlsx
20231219_NUT-60023_SettlementReport.xlsx
20231220_NUT-60023_SettlementReport.xlsx
20231221_NUT-60023_SettlementReport.xlsx
20231226_NUT-60023_SettlementReport.xlsx
20231227_NUT-60023_SettlementReport.xlsx
20231228_NUT-60023_SettlementReport.xlsx
20231231_NUT-60023_SettlementReport.xlsx
20240102_NUT-60023_SettlementReport.xlsx
20240103_NUT-60023_SettlementReport.xlsx
20240104_NUT-60023_SettlementReport.xlsx
20240107_NUT-60023_SettlementReport.xlsx
20240108_NUT-60023_SettlementReport.xlsx
20240109_NUT-600

In [33]:
## for backup
settdetail=settdetailcp.copy()

In [33]:
## run 12.3
settdetail=settdetail[settdetail['Dikirim oleh (fulfilled by)']=='FBB'].groupby(["Nomor pesanan (Order ID)",
                                                                                 'Nomor item pesanan (Order item ID)'])[['Nilai margin (Margin Amount)']].sum().reset_index().rename(columns = {'Nomor pesanan (Order ID)' : 'Order ID','Nomor item pesanan (Order item ID)' : 'Order Item ID'})

settlementmod=settlement.merge(settdetail,how='left',on=['Order ID','Order Item ID'])
settlementmod['Nilai margin (Margin Amount)']=settlementmod['Nilai margin (Margin Amount)'].fillna(0)
if len(settlementmod)==len(settlement):
    print('merge berhasil')
settlementmod['New Fee']=settlementmod['Fee']-settlementmod['Nilai margin (Margin Amount)']

merge berhasil


In [34]:
# run 12.35
if 'Old Fee' not in settlementmod.columns:
    print('replace done')
    settlementmod['Old Fee']=settlementmod['Fee']
    settlementmod['Fee']=settlementmod['New Fee']
    settlementmod['Fullfee']=settlementmod['Nilai margin (Margin Amount)']

    

replace done


In [35]:
# run 12.4
settlement=settlementmod.copy()

In [36]:
### run 13
blibli_merge = selcen_sku.copy()

In [311]:
selcen_sku[selcen_sku['No. Order']==12148247863]

,No. Order,No. Order Item,No. Awb,Tanggal Order,Nama Pemesan,Kode SKU,Blibli SKU,SKU,Product Name,Qty. Invoiced,...,Harga Cost 7,Bundle Name,Price List NFI,Total Net,Bundle Flag,Parent Item,Parent SKU,Harga Cost,Total Harga Cost,Total Net Before PPN


In [312]:
settlement[settlement['Order ID']=='12148247863'][['Order ID','Order Item ID','Fee','Payment Fee Merchant','Biaya Transaksi','Biaya Kirim Seller','Fullfee','Old Fee']]
# settlement[settlement['Biaya Transaksi']>0]
# settlement[settlement['Payment Fee Merchant']>0]
settlement[settlement['Transaction'] == 'Other Adjustments']

,Trx ID,Order ID,Order Item ID,Keterangan,Transaction,Delivered Date,Qty,Harga Satuan,Jumlah,Promosi Merchant,...,Biaya Transaksi,PPH 23,Biaya Kirim Seller,Total,Trx Source,Source,Nilai margin (Margin Amount),New Fee,Old Fee,Fullfee


In [37]:
### run 14
sales = settlement[settlement['Transaction'] == 'Sales']
cancel = settlement[settlement['Transaction'] == 'Cancel Sales']
other = settlement[settlement['Transaction'] == 'Other Adjustments']

cancel['Jumlah Cancel'] = cancel['Jumlah'].abs() * -1
cancel['Fee Cancel'] = cancel['Fee'].abs() * -1
other['Others'] = other['Total']
other['Other Adjustment'] = 'Yes'

sales = sales.merge(cancel[['Order ID', 'Order Item ID', 'Jumlah Cancel', 'Fee Cancel']], how = 'left', on = ['Order ID', 'Order Item ID']).merge(other[['Order ID', 'Order Item ID', 'Others', 'Other Adjustment']], how = 'left', on = ['Order ID', 'Order Item ID'])

settlement = sales.copy()
settlement['Jumlah'] = settlement['Jumlah'] - settlement['Jumlah Cancel'].fillna(0)
settlement['Fee'] = settlement['Fee'] - settlement['Fee Cancel'].fillna(0)

# settlement['Cek'] = settlement['Jumlah'] + settlement['Promosi Merchant'] - settlement['Fee'] - settlement['Payment Fee Merchant'] - settlement['Biaya Transaksi'] + settlement['PPH 23'] - settlement['Biaya Kirim Seller']
settlement['Cek'] = settlement['Jumlah'] + settlement['Promosi Merchant'] - settlement['Fee'] - settlement['Payment Fee Merchant'] - settlement['Biaya Transaksi'] + settlement['PPH 23'] - settlement['Biaya Kirim Seller']- settlement['Fullfee'] # feb 2024

settlement['Selisih'] = settlement['Cek'] - settlement['Total']
settlement['Bi Produk'] = settlement['Jumlah'] + settlement['Promosi Merchant']
settlement['Comfee'] = settlement['Fee']
# settlement['Others'] = settlement['Payment Fee Merchant'] + settlement['Biaya Transaksi'] + settlement['Biaya Kirim Seller'] + settlement['Others'].fillna(0)
settlement['Others'] = settlement['Biaya Kirim Seller']+settlement['Others'].fillna(0) ## di akhir other jadi shipping Fee Feb 2024
settlement['Payment Fee']= settlement['Payment Fee Merchant'] + settlement['Biaya Transaksi']
settlement['Fullfee']=settlement['Fullfee'].astype(int)

# display(settlement[settlement['Order ID']=='12148312199'][['Order ID','Order Item ID','Total','Jumlah','Cek','Promosi Merchant','Bi Produk', 'Comfee','Fullfee','Payment Fee', 'Others','Fee','Selisih']])


indeks = settlement[settlement['Selisih'] > 0].index.to_list()
settlement['Bi Produk'][indeks] = 0
settlement['Comfee'][indeks] = 0

settlement['Order ID'] = settlement['Order ID'].astype(str).str.replace('.0', '', regex = False)
settlement['Order Item ID'] = settlement['Order Item ID'].astype(str).str.replace('.0', '', regex = False)

settlement = settlement[settlement['Order ID'] != 'nan']
# set_pivot = pd.pivot_table(settlement, values = ['Bi Produk', 'Comfee', 'Others'], index = ['Order ID', 'Order Item ID', 'Keterangan'],  aggfunc=np.sum).reset_index()
set_pivot = settlement.groupby(['Order ID', 'Order Item ID', 'Keterangan']).agg({'Bi Produk':'sum', 'Comfee':'sum','Payment Fee':'sum', 'Others':'sum','Fullfee':'sum'}).reset_index()

set_pivot['Order ID'] = set_pivot['Order ID'].astype(str)
set_pivot['Order Item ID'] = set_pivot['Order Item ID'].astype(str)
blibli_merge['No. Order'] = blibli_merge['No. Order'].astype(str)
blibli_merge['No. Order Item'] = blibli_merge['No. Order Item'].astype(str)

no_prop = blibli_merge[blibli_merge['Brand'].isin(['Bonus Produk', 'Gimmick', 'Partnership'])]
blibli_prop = blibli_merge[~blibli_merge['Brand'].isin(['Bonus Produk', 'Gimmick', 'Partnership'])]

bundle_wms = blibli_prop[blibli_prop['Bundle Name'].notnull()]
blibli_set = blibli_prop[blibli_prop['Bundle Name'].isnull()]

duplicated = set_pivot[set_pivot[['Order ID', 'Keterangan']].duplicated(keep = False)]
not_duplicate = set_pivot[~set_pivot[['Order ID', 'Keterangan']].duplicated(keep = False)]

duplicated['Bi Produk'] = duplicated.groupby(['Order ID', 'Keterangan'])['Bi Produk'].transform('sum')
duplicated['Comfee'] = duplicated.groupby(['Order ID', 'Keterangan'])['Comfee'].transform('sum')

set_pivot = not_duplicate.append(duplicated, ignore_index = True, sort = False)

set_pivot = set_pivot.drop('Keterangan', axis = 1)

blibli_set = blibli_set.merge(set_pivot.drop_duplicates(['Order ID', 'Order Item ID']), how = 'left', left_on = ['No. Order', 'No. Order Item'], right_on = ['Order ID', 'Order Item ID'])
blibli_set['Bi Produk'] = blibli_set['Bi Produk'].fillna(0) * blibli_set['Total Net Before PPN']/blibli_set.groupby(['No. Order', 'Product Name'])['Total Net Before PPN'].transform('sum')
blibli_set['Comfee'] = blibli_set['Comfee'].fillna(0) * blibli_set['Total Net Before PPN']/blibli_set.groupby(['No. Order', 'Product Name'])['Total Net Before PPN'].transform('sum')
blibli_set['Others'] = blibli_set['Others'].fillna(0) * blibli_set['Total Net Before PPN']/blibli_set.groupby(['No. Order', 'Product Name'])['Total Net Before PPN'].transform('sum')
blibli_set['Payment Fee'] = blibli_set['Payment Fee'].fillna(0) * blibli_set['Total Net Before PPN']/blibli_set.groupby(['No. Order', 'Product Name'])['Total Net Before PPN'].transform('sum')##Feb 2024 
blibli_set['Fullfee'] = blibli_set['Fullfee'].fillna(0) * blibli_set['Total Net Before PPN']/blibli_set.groupby(['No. Order', 'Product Name'])['Total Net Before PPN'].transform('sum')##Feb 2024


bundle_set = bundle_wms.merge(blibli_set.groupby(['No. Order', 'Product Name'])[['Bi Produk', 'Comfee', 'Others','Payment Fee','Fullfee']].sum().reset_index(), how = 'left', left_on = ['No. Order', 'Bundle Name'], right_on = ['No. Order', 'Product Name'])
bundle_set = bundle_set.rename(columns = {'Product Name_x' : 'Product Name'})
bundle_set = bundle_set.drop('Product Name_y', axis = 1)

bundle_set['Bi Produk'] = bundle_set['Bi Produk'].fillna(0) * bundle_set['Total Net Before PPN']/bundle_set.groupby(['No. Order', 'Bundle Name'])['Total Net Before PPN'].transform('sum')
bundle_set['Comfee'] = bundle_set['Comfee'].fillna(0) * bundle_set['Total Net Before PPN']/bundle_set.groupby(['No. Order', 'Bundle Name'])['Total Net Before PPN'].transform('sum')
bundle_set['Others'] = bundle_set['Others'].fillna(0) * bundle_set['Total Net Before PPN']/bundle_set.groupby(['No. Order', 'Bundle Name'])['Total Net Before PPN'].transform('sum')
bundle_set['Payment Fee'] = bundle_set['Payment Fee'].fillna(0) * bundle_set['Total Net Before PPN']/bundle_set.groupby(['No. Order', 'Product Name'])['Total Net Before PPN'].transform('sum')##Feb 2024 
bundle_set['Fullfee'] = bundle_set['Fullfee'].fillna(0) * bundle_set['Total Net Before PPN']/bundle_set.groupby(['No. Order', 'Product Name'])['Total Net Before PPN'].transform('sum')##Feb 2024

blibli_set = blibli_set.append(bundle_set, ignore_index = True, sort = False)
blibli_set = blibli_set.append(no_prop, ignore_index = True, sort = False)

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [ ]:
blibli_set[(blibli_set['Bi Produk'].isnull())
            & (blibli_set['Bundle Name'].isnull()) &
            (blibli_set['No. Order'].isin(settlement['Order ID']))].groupby(['Real Nama Produk', 'Real SKU', 'Bundle Name', 'Qty. Invoiced'], dropna = False)['No. Order'].first().reset_index()#['No. Order'].unique()
### cek aja apa semua bener krn qty invoiced 0 di selcen liat Order Status

selcen[(selcen["No. Order"]=='12091471690')]





In [262]:
blibli_set[blibli_set['No. Order']==12148247863]

,No. Order,No. Order Item,No. Awb,Tanggal Order,Nama Pemesan,Kode SKU,Blibli SKU,SKU,Product Name,Qty. Invoiced,...,Harga Cost,Total Harga Cost,Total Net Before PPN,Order ID,Order Item ID,Bi Produk,Comfee,Payment Fee,Others,Fullfee


In [38]:
### run 14
cancel['Order Item ID'] = cancel['Order Item ID'].astype(str).str.replace('.0', '', regex = False)
other['Order Item ID'] = other['Order Item ID'].astype(str).str.replace('.0', '', regex = False)

bli_payment = blibli_set[blibli_set['Bundle Name'].isnull()]
bundle_payment = blibli_set[blibli_set['Bundle Name'].notnull()]

bli_payment['Paid'] = np.nan
indeks = bli_payment[bli_payment['Order Item ID'].isin(settlement['Order Item ID'])].index.to_list()
bli_payment['Paid'][indeks] = 'Paid'
bli_payment['Paid'] = bli_payment['Paid'].fillna('Unpaid')

indeks = bli_payment[(bli_payment['Order Item ID'].isin(cancel['Order Item ID'])) & 
                    (bli_payment['Bi Produk'] > 0)].index.to_list()
bli_payment['Paid'][indeks] = 'Half Paid'

indeks = bli_payment[(bli_payment['Order Item ID'].isin(cancel['Order Item ID'])) & 
                    (bli_payment['Bi Produk'] == 0) & (bli_payment['Paid'] == 'Unpaid')].index.to_list()
bli_payment['Paid'][indeks] = 'Cancel'

indeks = bli_payment[(bli_payment['Order Item ID'].isin(other['Order Item ID'])) & 
                    (bli_payment['Bi Produk'] > 0)].index.to_list()
bli_payment['Paid'][indeks] = 'Half Paid'

indeks = bli_payment[(bli_payment['Order Status'] == 'Order Batal')  & (bli_payment['Paid'] == 'Unpaid')].index.to_list()
bli_payment['Paid'][indeks] = 'Cancel'

# indeks = bli_payment[(bli_payment['Order Status'] == 'Pengiriman Gagal') & 
#                     (bli_payment['Bi Produk'] > 0) ].index.to_list()
# bli_payment['Paid'][indeks] = 'Half Paid'

indeks = bli_payment[(bli_payment['Order Status'] == 'Pengiriman Gagal') & 
                    (bli_payment['Bi Produk'] == 0) & (bli_payment['Paid'] == 'Unpaid')].index.to_list()
bli_payment['Paid'][indeks] = 'Cancel'

bundle_payment = bundle_payment.merge(bli_payment.groupby(['No. Order', 'Product Name'])[['Paid']].first().reset_index(), how = 'left', left_on = ['No. Order', 'Bundle Name'], right_on = ['No. Order', 'Product Name'])
bundle_payment = bundle_payment.rename(columns = {'Product Name_x' : 'Product Name'})
bundle_payment = bundle_payment.drop('Product Name_y', axis = 1)

bli_payment = bli_payment.append(bundle_payment, ignore_index = True, sort = False)

bli_payment_gimmick = bli_payment[bli_payment['Brand'].isin(['Gimmick', 'Bonus Produk'])]
paid = bli_payment[bli_payment['Paid'] == 'Paid']['No. Order'].unique()

indeks = bli_payment_gimmick[bli_payment_gimmick['No. Order'].isin(paid)].index.to_list()
bli_payment['Paid'][indeks] = 'Paid'

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\pandas\core\series.py:1135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

S

In [39]:
### run 15
bli_payment['Bi Produk Adj'] = bli_payment['Bi Produk']/1.11
bli_payment['BTL Cost'] = bli_payment['Bi Produk Adj'] - bli_payment['Total Net Before PPN']

In [40]:
### run 16
tambah_unpaid = pd.read_excel(r'Settlement Project/unpaid order to Andra.xlsx', sheet_name = 'Unpaid to cancel or not found')
tambah_unpaid2 = pd.read_excel(r'Settlement Project/Unpaid Mar.xlsx', sheet_name = 'Blibli NM')

tambah_unpaid = tambah_unpaid.append(tambah_unpaid2, ignore_index = True, sort = False)
tambah_unpaid['No. Order'] = tambah_unpaid['Order #'].astype(str)
indeks = bli_payment[bli_payment['No. Order'].isin(tambah_unpaid['Order #'])].index.to_list()
print(len(indeks))
bli_payment['Order Status'][indeks] = 'Cancel'
bli_payment['Paid'][indeks] = 'Cancel'

rumus_status = pd.read_excel(r'Settlement Project/Rumus Status.xlsx', sheet_name = 'Blibli NM')
if len(rumus_status != 0):
    for i in range(len(rumus_status)):
        stat = rumus_status['Order Status'][i]
        indeks = bli_payment[(bli_payment['Paid'] == 'Unpaid') &
                             (bli_payment['Order Status'].str.lower() == stat.lower())].index.to_list()
        bli_payment['Order Status'][indeks] = 'Cancel'
        bli_payment['Paid'][indeks] = 'Cancel'

0


C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentati

In [51]:
#### run 17
total_trx_full = bli_payment.rename(columns = {
    'No. Order Item' : 'Product ID',
    'Total Bayar' : 'Total Selcen',
    'Status Seller Centre' : 'Order Status_y',
    'Toko/Gudang' : 'Warehouse Origin',
    'Bi Produk' : 'Total Produk Prop',
    'Comfee' : 'Comfee Prop',
    'Tanggal Order' : 'Order Date Seller Centre',
    'Bi Produk Adj' : 'Bi Bahan Produk Final Adj', 
    'Outbound Qty' : 'Warehouse Quantity'
})

total_trx_full['WH'] = total_trx_full['Warehouse Origin']
total_trx_full['SO STATUS'] = total_trx_full['Order Status']

total_trx_full = total_trx_full.rename(columns = {
    'Total Selcen' : 'Total Seller Centre', 
    'Order Status_y' : 'Order Status Seller Centre',
    'SO STATUS' : 'Status Gudang',
    'Shipment Fee' : 'Shipping Fee', 
    'Bi Bahan Produk Final Adj' : 'Total Produk Prop Adj'
})

In [52]:
### run 18
total_trx_full['Duplicated'] = np.nan

dups_order = settlement[(settlement[['Order ID', 'Order Item ID', 'Keterangan', 'Transaction']].duplicated(keep = False)) & 
          (settlement['Order Item ID'].notnull()) & 
          (settlement['Order Item ID'] != 'nan')]['Order ID'].unique()
indeks = total_trx_full[total_trx_full['No. Order'].isin(dups_order)].index.to_list()
total_trx_full['Duplicated'][indeks] = 'Duplicated Order'

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [53]:
### run 19

source['Order #'] = source['Order #'].astype(str)

total_trx_full['Bundle Name'] = total_trx_full['Bundle Name'].astype(str).str.strip().str.replace('\\n', '', regex = False).str.replace('\n', '', regex = False)
source['Bundle Name'] = source['Bundle Name'].astype(str).str.strip().str.replace('\\n', '', regex = False).str.replace('\n', '', regex = False)


dict_change = {'Buy 12 Get 12 ? HiLo School Chocolate RTD 200ml (ED 6 bulan)':'Buy 12 Get 12 – HiLo School Chocolate RTD 200ml (ED 6 bulan)'}

for key in dict_change:
    total_trx_full['Bundle Name'] = total_trx_full['Bundle Name'].astype(str).str.replace(key, dict_change[key], regex = False)
    source['Bundle Name'] = source['Bundle Name'].astype(str).str.replace(key, dict_change[key], regex = False)
    
               
total_trx_merge = total_trx_full.merge(source[['Order #', 'Real SKU', 'Real Nama Produk', 'Bundle Name', 'Qty. Invoiced', 'Total Net', 'Total Produk Prop', 'Comfee Prop']].drop_duplicates(['Order #', 'Real SKU', 'Real Nama Produk', 'Bundle Name', 'Qty. Invoiced']), how = 'left', left_on = ['No. Order', 'Real SKU', 'Real Nama Produk', 'Bundle Name', 'Qty. Invoiced'], right_on = ['Order #', 'Real SKU', 'Real Nama Produk', 'Bundle Name', 'Qty. Invoiced'])

total_trx_null = total_trx_merge[total_trx_merge['Total Net_y'].isnull()]

total_trx_null = total_trx_null.drop(['Order #', 'Total Net_y', 'Total Produk Prop_y', 'Comfee Prop_y'], axis = 1)
total_trx_null = total_trx_null.rename(columns = {'Total Net_x' : 'Total Net', 
                                                  'Total Produk Prop_x' : 'Total Produk Prop',
                                                  'Comfee Prop_x' : 'Comfee Prop'})

total_trx_null = total_trx_null.merge(source[['Order #', 'Product Name', 'Bundle Name', 'Total Net', 'Total Produk Prop', 'Comfee Prop']].drop_duplicates(['Order #', 'Product Name', 'Bundle Name']), how = 'left', left_on = ['No. Order', 'Product Name',  'Bundle Name'], right_on = ['Order #', 'Product Name', 'Bundle Name']).set_index(total_trx_null.index) 
total_trx_merge['Total Net_y'][total_trx_null.index] = total_trx_null['Total Net_y']
total_trx_merge['Total Produk Prop_y'][total_trx_null.index] = total_trx_null['Total Produk Prop_y']
total_trx_merge['Comfee Prop_y'][total_trx_null.index] = total_trx_null['Comfee Prop_y']

total_trx_null = total_trx_merge[total_trx_merge['Total Net_y'].isnull()]

total_trx_null = total_trx_null.drop(['Order #', 'Total Net_y', 'Total Produk Prop_y', 'Comfee Prop_y'], axis = 1)
total_trx_null = total_trx_null.rename(columns = {'Total Net_x' : 'Total Net',
                                                  'Total Produk Prop_x' : 'Total Produk Prop',
                                                  'Comfee Prop_x' : 'Comfee Prop'})

total_trx_null = total_trx_null.merge(source[['Order #', 'Real SKU', 'Bundle Name', 'Total Net', 'Total Produk Prop', 'Comfee Prop']].drop_duplicates(['Order #', 'Real SKU', 'Bundle Name']), how = 'left', left_on = ['No. Order', 'Real SKU',  'Bundle Name'], right_on = ['Order #', 'Real SKU', 'Bundle Name']).set_index(total_trx_null.index) 
total_trx_merge['Total Net_y'][total_trx_null.index] = total_trx_null['Total Net_y']
total_trx_merge['Total Produk Prop_y'][total_trx_null.index] = total_trx_null['Total Produk Prop_y']
total_trx_merge['Comfee Prop_y'][total_trx_null.index] = total_trx_null['Comfee Prop_y']

total_trx_null = total_trx_merge[total_trx_merge['Total Net_y'].isnull()]

# source2 = source.copy()

# source2['Real SKU'] = source2['Real SKU'].astype(str).str.replace('2309005305', '2309005300')

# # indeks = source2[source2['Real SKU'] == '(B)2101809250'].index.to_list()
# # source2['Real SKU'][indeks] = '(U)2101809250'

# # indeks = source2[source2['Real SKU'] == 'PH48(3)B35'].index.to_list()
# # source2['Real SKU'][indeks] = '2101941250P4'

# # indeks = source2[source2['Real SKU'] == 'PH39(3)B17'].index.to_list()
# # source2['Real SKU'][indeks] = '2101809250P4'

# total_trx_null = total_trx_null.drop(['Order #', 'Total Net_y', 'Total Produk Prop_y', 'Comfee Prop_y'], axis = 1)
# total_trx_null = total_trx_null.rename(columns = {'Total Net_x' : 'Total Net', 
#                                                   'Total Produk Prop_x' : 'Total Produk Prop',
#                                                   'Comfee Prop_x' : 'Comfee Prop'})

# total_trx_null = total_trx_null.merge(source[['Order #', 'Product Name', 'Bundle Name', 'Total Net', 'Total Produk Prop', 'Comfee Prop']].drop_duplicates(['Order #', 'Product Name', 'Bundle Name']), how = 'left', left_on = ['No. Order', 'Product Name',  'Bundle Name'], right_on = ['Order #', 'Product Name', 'Bundle Name']).set_index(total_trx_null.index) 
# total_trx_merge['Total Net_y'][total_trx_null.index] = total_trx_null['Total Net_y']
# total_trx_merge['Total Produk Prop_y'][total_trx_null.index] = total_trx_null['Total Produk Prop_y']
# total_trx_merge['Comfee Prop_y'][total_trx_null.index] = total_trx_null['Comfee Prop_y']

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [46]:
total_trx_null[total_trx_null['No. Order'].isin(source['Order #'])][['No. Order', 'Real SKU', 'Real Nama Produk', 'Bundle Name', 'Paid', 'Total Net_x', 'Total Net_y']].drop_duplicates(['Real SKU', 'Real Nama Produk', 'Bundle Name'])

,No. Order,Real SKU,Real Nama Produk,Bundle Name,Paid,Total Net_x,Total Net_y


In [54]:
### run 20

total_trx_merge = total_trx_merge.rename(columns = {'Total Produk Prop_x' : 'Total Produk Prop by Selcen', 
                                                   'Total Produk Prop_y' : 'Total Produk Prop Real Jan',
                                                   'Total Net_x' : 'Total Net by Selcen', 
                                                   'Total Net_y' : 'Total Net Real Jan',
                                                   'Fullfee Prop_x' : 'Fullfee Prop by Selcen', 
                                                   'Fullfee Prop_y' : 'Fullfee Prop Real Jan',
                                                   'Comfee Prop_x' : 'Comfee Prop by Selcen', 
                                                   'Comfee Prop_y' : 'Comfee Prop Real Jan'})

total_trx_merge['Delta Total Net'] = total_trx_merge['Total Net by Selcen'] - total_trx_merge['Total Net Real Jan']
total_trx_merge['Delta Total Produk Prop'] = total_trx_merge['Total Produk Prop by Selcen'] - total_trx_merge['Total Produk Prop Real Jan']
total_trx_merge['Delta Comfee Prop'] = total_trx_merge['Comfee Prop by Selcen'] - total_trx_merge['Comfee Prop Real Jan']

In [ ]:
# total_trx_merge[total_trx_merge['No. Order'].isin(source[source['Paid January'] == 'Paid']['Order #'])].to_csv(r'D:\Backup\Backup 26 Mei\Settlement Project Backup/Blibli Testing by Selcen Compare.csv', sep = ';', index = False)
total_trx_merge[total_trx_merge['No. Order']=='12148247863']

In [55]:
## run21 - ganti nama folder

from datetime import date
total_trx_merge.to_csv(r'Settlement Project\Backup\Realisasi Aug 2024\Final Result\Before Compare/Blibli Testing by Selcen Compare v2.csv', sep = ';', index = False)

In [383]:
# total_trx_merge[(total_trx_merge['Year']==2023)&(total_trx_merge['Pemilik Budget'].notnull())]#['Pemilik Budget']

In [48]:
a='11'
"2222 " + a

'2222 11'

In [55]:
### only for diplay
pd.options.display.max_columns = 150

In [56]:
total_trx_merge[total_trx_merge['Year']==2023]['Month'].unique()

array(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August', 'September', 'October', 'November', 'December'],
      dtype=object)

In [56]:
### run 22
import pandas as pd
import numpy as np
import os
total_trx=total_trx_merge.copy()
# total_trx = pd.read_csv(r'Settlement Project\Backup\Realisasi Dec 2023\Final Result\Before Compare/Blibli Testing by Selcen Compare v2.csv', sep = ';')

total_trx_full = total_trx.rename(columns = {'Total Produk Prop by Selcen': 'Total Produk Prop', 
                                                   'Total Net by Selcen' : 'Total Net', 
                                                   'Fullfee Prop by Selcen' : 'Fullfee Prop', 
                                                   'Comfee Prop by Selcen' : 'Comfee Prop'})


total_trx_full = total_trx_full.drop(['Order #', 'Total Net Real Jan', 'Total Produk Prop Real Jan',
       'Comfee Prop Real Jan', 'Delta Total Net', 'Delta Total Produk Prop',
       'Delta Comfee Prop'], axis = 1)

source = pd.read_csv(r"D:\6. settlement\Code\Settlement Project\Backup\Realisasi Jul 2024\Final Result\Raw Unpivoted\CSV\Runningan Blibli By Selcen Jan 22 - Jul 24.csv", sep = ';')

prevmonth='July'
# source['True datetime'] = pd.to_datetime(source['True datetime'])
source = source[(source['Year'] > 2022)]

source['Order #'] = source['No. Order']#.astype(str)
source['Order #'] = source['Order #'].astype(str)
source['Real SKU'] = source['Real SKU'].astype(str)
total_trx_full['No. Order'] = total_trx_full['No. Order'].astype(str)

# selcen_old = selcen_sku[selcen_sku['OrderId'].isin(source['Order #'])]

source['Product Name'] = source['Product Name'].fillna(source['Real Nama Produk'])

source['Order #'] = source['Order #'].astype(str)

total_trx_full['Bundle Name'] = total_trx_full['Bundle Name'].astype(str).str.strip().str.replace('\\n', '', regex = False).str.replace('\n', '', regex = False)
source['Bundle Name'] = source['Bundle Name'].astype(str).str.strip().str.replace('\\n', '', regex = False).str.replace('\n', '', regex = False)


dict_change = {'Buy 12 Get 12 ? HiLo School Chocolate RTD 200ml (ED 6 bulan)':'Buy 12 Get 12 – HiLo School Chocolate RTD 200ml (ED 6 bulan)'}

for key in dict_change:
    total_trx_full['Bundle Name'] = total_trx_full['Bundle Name'].astype(str).str.replace(key, dict_change[key], regex = False)
    source['Bundle Name'] = source['Bundle Name'].astype(str).str.replace(key, dict_change[key], regex = False)
    
               
total_trx_merge = total_trx_full.merge(source[['Order #', 'Real SKU', 'Real Nama Produk', 'Bundle Name', 'Paid '+prevmonth,'Qty. Invoiced', 'Total Net', 'Total Produk Prop', 'Comfee Prop']].drop_duplicates(['Order #', 'Real SKU', 'Real Nama Produk', 'Bundle Name', 'Qty. Invoiced']), how = 'left', left_on = ['No. Order', 'Real SKU', 'Real Nama Produk', 'Bundle Name', 'Qty. Invoiced'], right_on = ['Order #', 'Real SKU', 'Real Nama Produk', 'Bundle Name', 'Qty. Invoiced'])

total_trx_null = total_trx_merge[total_trx_merge['Total Net_y'].isnull()]

total_trx_null = total_trx_null.drop(['Order #', 'Paid '+prevmonth,'Total Net_y', 'Total Produk Prop_y', 'Comfee Prop_y'], axis = 1)
total_trx_null = total_trx_null.rename(columns = {'Total Net_x' : 'Total Net', 
                                                  'Total Produk Prop_x' : 'Total Produk Prop',
                                                  'Comfee Prop_x' : 'Comfee Prop'})

total_trx_null = total_trx_null.merge(source[['Order #', 'Product Name', 'Bundle Name', 'Paid '+prevmonth,'Total Net', 'Total Produk Prop', 'Comfee Prop']].drop_duplicates(['Order #', 'Product Name', 'Bundle Name']), how = 'left', left_on = ['No. Order', 'Product Name',  'Bundle Name'], right_on = ['Order #', 'Product Name', 'Bundle Name']).set_index(total_trx_null.index) 
total_trx_merge['Total Net_y'][total_trx_null.index] = total_trx_null['Total Net_y']
total_trx_merge['Total Produk Prop_y'][total_trx_null.index] = total_trx_null['Total Produk Prop_y']
total_trx_merge['Comfee Prop_y'][total_trx_null.index] = total_trx_null['Comfee Prop_y']
total_trx_merge['Paid '+prevmonth][total_trx_null.index] = total_trx_null['Paid '+prevmonth]

total_trx_null = total_trx_merge[total_trx_merge['Total Net_y'].isnull()]

total_trx_null = total_trx_null.drop(['Order #', 'Paid '+prevmonth,'Total Net_y', 'Total Produk Prop_y', 'Comfee Prop_y'], axis = 1)
total_trx_null = total_trx_null.rename(columns = {'Total Net_x' : 'Total Net',
                                                  'Total Produk Prop_x' : 'Total Produk Prop',
                                                  'Comfee Prop_x' : 'Comfee Prop'})

total_trx_null = total_trx_null.merge(source[['Order #', 'Real SKU', 'Bundle Name', 'Paid '+prevmonth,'Total Net', 'Total Produk Prop', 'Comfee Prop']].drop_duplicates(['Order #', 'Real SKU', 'Bundle Name']), how = 'left', left_on = ['No. Order', 'Real SKU',  'Bundle Name'], right_on = ['Order #', 'Real SKU', 'Bundle Name']).set_index(total_trx_null.index) 
total_trx_merge['Total Net_y'][total_trx_null.index] = total_trx_null['Total Net_y']
total_trx_merge['Total Produk Prop_y'][total_trx_null.index] = total_trx_null['Total Produk Prop_y']
total_trx_merge['Comfee Prop_y'][total_trx_null.index] = total_trx_null['Comfee Prop_y']
total_trx_merge['Paid '+prevmonth][total_trx_null.index] = total_trx_null['Paid '+prevmonth]

total_trx_null = total_trx_merge[total_trx_merge['Total Net_y'].isnull()]

# source2 = source.copy()

# source2['Real SKU'] = source2['Real SKU'].astype(str).str.replace('2309005305', '2309005300')

# # indeks = source2[source2['Real SKU'] == '(B)2101809250'].index.to_list()
# # source2['Real SKU'][indeks] = '(U)2101809250'

# # indeks = source2[source2['Real SKU'] == 'PH48(3)B35'].index.to_list()
# # source2['Real SKU'][indeks] = '2101941250P4'

# # indeks = source2[source2['Real SKU'] == 'PH39(3)B17'].index.to_list()
# # source2['Real SKU'][indeks] = '2101809250P4'

# total_trx_null = total_trx_null.drop(['Order #', 'Total Net_y', 'Total Produk Prop_y', 'Comfee Prop_y'], axis = 1)
# total_trx_null = total_trx_null.rename(columns = {'Total Net_x' : 'Total Net', 
#                                                   'Total Produk Prop_x' : 'Total Produk Prop',
#                                                   'Comfee Prop_x' : 'Comfee Prop'})

# total_trx_null = total_trx_null.merge(source[['Order #', 'Product Name', 'Bundle Name', 'Total Net', 'Total Produk Prop', 'Comfee Prop']].drop_duplicates(['Order #', 'Product Name', 'Bundle Name']), how = 'left', left_on = ['No. Order', 'Product Name',  'Bundle Name'], right_on = ['Order #', 'Product Name', 'Bundle Name']).set_index(total_trx_null.index) 
# total_trx_merge['Total Net_y'][total_trx_null.index] = total_trx_null['Total Net_y']
# total_trx_merge['Total Produk Prop_y'][total_trx_null.index] = total_trx_null['Total Produk Prop_y']
# total_trx_merge['Comfee Prop_y'][total_trx_null.index] = total_trx_null['Comfee Prop_y']

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (15,54,60,67,74,78,90) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\timotius.giovand

In [50]:
# total_trx_null[total_trx_null['No. Order'].isin(source['Order #'])][['No. Order', 'Real SKU', 'Real Nama Produk', 'Bundle Name', 'Paid', 'Total Net_x', 'Total Net_y']].drop_duplicates(['Real SKU', 'Real Nama Produk', 'Bundle Name'])

In [57]:
### run 21
total_trx_merge['Paid Beda'] = np.nan

indeks = total_trx_merge[total_trx_merge['Paid'] != total_trx_merge['Paid '+prevmonth]].index.to_list()
total_trx_merge['Paid Beda'][indeks] = 'Beda'

total_trx_merge['Paid Beda'] = total_trx_merge['Paid Beda'].fillna('Sama')
# total_trx_merge[total_trx_merge['Paid Beda'] == 'Beda'][['Order #','Brand','Real SKU','Month', 'Paid_x', 'Paid_y', 'Paid Beda']]
total_trx_merge[total_trx_merge['Paid Beda'] == 'Beda'].groupby(['Month', 'Paid', 'Paid '+prevmonth], dropna = False)['No. Order'].nunique()

C:\Users\timotius.giovandi\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Month   Paid    Paid July
August  Cancel  NaN            49
        Paid    NaN          1221
        Unpaid  NaN            91
July    Cancel  Unpaid          1
                NaN            55
        Paid    Unpaid        204
                NaN           926
Name: No. Order, dtype: int64

In [54]:
## yg sebelumnya paid jadi unpaid 
## terakhir isunya salah di loop load settlement
# total_trx_merge[(total_trx_merge['Paid '+prevmonth].isnull())&
#                 (total_trx_merge['Paid']=='Paid')&(total_trx_merge['Month']=='June')][['No. Order','Year','Month','Date','Real SKU']].sort_values('Date')
                                                                                                                                            
# total_trx[total_trx['Year']==2023]['No. Order'].nunique()
# source[source['No. Order']==12127025707]

In [58]:
### run 22
total_trx_merge = total_trx_merge.rename(columns = {'Paid' : 'Paid August'})

In [59]:
### run 23
total_trx_merge = total_trx_merge.rename(columns = {'Total Produk Prop_x' : 'Total Produk Prop by Selcen', 
                                                   'Total Produk Prop_y' : 'Total Produk Prop Real Jan',
                                                   'Total Net_x' : 'Total Net by Selcen', 
                                                   'Total Net_y' : 'Total Net Real Jan',
                                                   'Fullfee Prop_x' : 'Fullfee Prop by Selcen', 
                                                   'Fullfee Prop_y' : 'Fullfee Prop Real Jan',
                                                   'Comfee Prop_x' : 'Comfee Prop by Selcen', 
                                                   'Comfee Prop_y' : 'Comfee Prop Real Jan'})

total_trx_merge['Delta Total Net'] = total_trx_merge['Total Net by Selcen'] - total_trx_merge['Total Net Real Jan']
total_trx_merge['Delta Total Produk Prop'] = total_trx_merge['Total Produk Prop by Selcen'] - total_trx_merge['Total Produk Prop Real Jan']
total_trx_merge['Delta Comfee Prop'] = total_trx_merge['Comfee Prop by Selcen'] - total_trx_merge['Comfee Prop Real Jan']

In [60]:
### run 24
total_trx_merge = total_trx_merge.rename(columns = {'Total Produk Prop by Selcen': 'Total Produk Prop', 
                                                   'Total Net by Selcen' : 'Total Net', 
                                                   'Fullfee Prop by Selcen' : 'Fullfee Prop', 
                                                    'Fullfee' : 'Fullfee Prop', 
                                                    'Payment Fee' : 'Payment Fee Prop', 
                                                    'Others': 'Shipping Fee Prop',
                                                   'Comfee Prop by Selcen' : 'Comfee Prop'})


total_trx_merge = total_trx_merge.drop(['Order #', 'Total Net Real Jan', 'Total Produk Prop Real Jan',
       'Comfee Prop Real Jan', 'Delta Total Net', 'Delta Total Produk Prop',
       'Delta Comfee Prop'], axis = 1)

In [61]:
total_trx_merge = total_trx_merge.rename(columns = {'Others': 'Shipping Fee Prop'})

In [62]:
# total_trx_merge[(total_trx_merge['No. Order']=='12148275965')&
#                 (total_trx_merge['Brand']!='Bundle')].groupby(['Year','Month','No. Order'])[['Total Net Before PPN','Total Produk Prop Adj','BTL Cost','Comfee Prop','Shipping Fee Prop','Payment Fee Prop','Fullfee Prop']].sum().reset_index().columns
total_trx_merge['Disc 2 Non Faktur Pajak']=0
total_trx_merge['Disc 2 Faktur Pajak']=0


In [63]:
total_trx_merge['Disc 2 Non Faktur Pajak']=total_trx_merge['Shipping Fee Prop']+total_trx_merge['Payment Fee Prop']
total_trx_merge['Disc 2 Faktur Pajak']=total_trx_merge['Fullfee Prop']


In [64]:
### run 25
hasilakhir=pd.read_excel("Settlement Project\Backup\Realisasi Apr 2023\Final Result\Raw Unpivoted\Excel\Runningan Blibli By Selcen Jan 22 - Apr 23.xlsx")
# hasilakhir=hasilakhir.rename(columns={'Paid February':'Paid March 1'})
hasilakhir=hasilakhir.rename(columns={'Paid April':'Paid August'})
hasilakhir=hasilakhir.rename(columns={'Paid March':'Paid July'})


hasilakhir=hasilakhir.head(0)
hasilakhir['Pemilik Budget']=np.nan
# hasilakhir=hasilakhir.append(total_trx_merge)
hasilakhir=total_trx_merge[[x for x in total_trx_merge.columns if x in hasilakhir.columns]+['Shipping Fee Prop','Payment Fee Prop', 'Fullfee Prop','Disc 2 Faktur Pajak','Disc 2 Non Faktur Pajak']]


In [65]:
## run 26
hasilakhir.to_csv(r'Settlement Project\Backup\Realisasi Aug 2024\Final Result\Raw Unpivoted\CSV/Runningan Blibli By Selcen Jan 22 - Aug 24.csv', sep = ';', index = False)
hasilakhir.to_excel(r'Settlement Project\Backup\Realisasi Aug 2024\Final Result\Raw Unpivoted\Excel/Runningan Blibli By Selcen Jan 22 - Aug 24.xlsx', index = False)

In [ ]:
#### selesai :) #####